# Szeregi czasowe
## DatetimeIndex

In [ ]:
# Zacznijmy od zaimportowania pakietów, których użyjemy w tym rozdziale
# oraz od ustawienia silnika tworzenia wykresów na Plotly.
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"

In [ ]:
# To tworzy DatetimeIndex na podstawie znacznika czasu początkowego,
# liczby okresów i częstotliwości ("D" = dzienna).
daily_index = pd.date_range("2020-02-28", periods=4, freq="D")
daily_index

In [ ]:
# To tworzy DatetimeIndex na podstawie znaczników czasowych początku i końca.
# Częstotliwość jest ustawiona na "co tydzień w niedziele" ("W-SUN").
weekly_index = pd.date_range("2020-01-01", "2020-01-31", freq="W-SUN")
weekly_index

In [ ]:
# Utworzenie DataFrame na podstawie weekly_index. Może to być liczba
# odwiedzających muzeum, które jest otwarte tylko w niedziele.
pd.DataFrame(data=[21, 15, 33, 34],
             columns=["odwiedzający"], index=weekly_index)

In [ ]:
msft = pd.read_csv("csv/MSFT.csv")

In [ ]:
msft.info()

In [ ]:
msft.loc[:, "Date"] = pd.to_datetime(msft["Date"])

In [ ]:
msft.dtypes

In [ ]:
msft = pd.read_csv("csv/MSFT.csv",
                   index_col="Date", parse_dates=["Date"])

In [ ]:
msft.info()

In [ ]:
msft.loc[:, "Volume"] = msft["Volume"].astype("float")
msft["Volume"].dtype

In [ ]:
msft = msft.sort_index()

In [ ]:
msft.index.date

In [ ]:
msft.loc["2019", "Adj Close"]

In [ ]:
msft.loc["2019-06":"2020-05", "Adj Close"].plot()

## Praca ze strefami czasowymi

In [ ]:
# Dodanie do daty informacji o godzinie
msft_close = msft.loc[:, ["Adj Close"]].copy()
msft_close.index = msft_close.index + pd.DateOffset(hours=16)
msft_close.head(2)

In [ ]:
# Uwzględnienie strefy czasowej w znacznikach czasu
msft_close = msft_close.tz_localize("America/New_York")
msft_close.head(2)

In [ ]:
msft_close = msft_close.tz_convert("UTC")
msft_close.loc["2020-01-02", "Adj Close"]  # 21:00 bez DST

In [ ]:
msft_close.loc["2020-05-01", "Adj Close"]  # 20:00 z DST

## Przesunięcia i zmiany procentowe

In [ ]:
msft_close.head()

In [ ]:
msft_close.shift(1).head()

In [ ]:
returns = np.log(msft_close / msft_close.shift(1))
returns = returns.rename(columns={"Adj Close": "Stopa zwrotu"})
returns.head()

In [ ]:
# Wykreślenie histogramu z dziennymi logarytmicznymi stopami zwrotu
returns.plot.hist()

In [ ]:
simple_rets = msft_close.pct_change()
simple_rets = simple_rets.rename(columns={"Adj Close": "Prosta stopa zwrotu"})
simple_rets.head()

## Zmiana bazy i korelacja

In [ ]:
parts = []  # Lista do zebrania poszczególnych DataFrame
for ticker in ["AAPL", "AMZN", "GOOGL", "MSFT"]:
    # "usecols" umożliwia nam wczytanie tylko kolumn Date i Adj Close
    adj_close = pd.read_csv(f"csv/{ticker}.csv",
                            index_col="Date", parse_dates=["Date"],
                            usecols=["Date", "Adj Close"])
    # Zmiana nazwy kolumny na symbol giełdowy
    adj_close = adj_close.rename(columns={"Adj Close": ticker})
    # Dołączanie DataFrame z akcjami do listy parts
    parts.append(adj_close)

In [ ]:
# Połączenie czterech obiektów DataFrame w jeden
adj_close = pd.concat(parts, axis=1)
adj_close

In [ ]:
adj_close = adj_close.dropna()
adj_close.info()

In [ ]:
# Użyjemy próbki z okresu czerwiec 2019 - maj 2020
adj_close_sample = adj_close.loc["2019-06":"2020-05", :]
rebased_prices = adj_close_sample / adj_close_sample.iloc[0, :] * 100
rebased_prices.head(2)

In [ ]:
rebased_prices.plot()

In [ ]:
# Korelacja dziennych logarytmicznych stóp zwrotu
returns = np.log(adj_close / adj_close.shift(1))
returns.corr()

In [ ]:
import plotly.express as px

In [ ]:
fig = px.imshow(returns.corr(),
                x=adj_close.columns,
                y=adj_close.columns,
                color_continuous_scale=list(
                    reversed(px.colors.sequential.RdBu)),
                zmin=-1, zmax=1)
fig.show()

## Resampling

In [ ]:
end_of_month = adj_close.resample("M").last()
end_of_month.head()

In [ ]:
end_of_month.resample("D").asfreq().head()  # Bez transformacji

In [ ]:
end_of_month.resample("W-FRI").ffill().head()  # Uzupełnianie wartości

## Okna kroczące

In [ ]:
# Wyznaczenie średniej kroczącej dla MSFT z danymi z 2019 r.
msft19 = msft.loc["2019", ["Adj Close"]].copy()
# Dodanie 25-dniowej średniej kroczącej jako nowej kolumny DataFrame
msft19.loc[:, "średnia z 25 dni"] = msft19["Adj Close"].rolling(25).mean()
msft19.plot()